In [ ]:
import numpy as np
import batoid
import batoid_rubin
import ipywidgets
from pathlib import Path

In [ ]:
# We need to download the bending mode and FEA data
# This cell will download it to your home directory.
home_dir = Path.home()
data_dir = home_dir / ".local" / "batoid_rubin_data"

from batoid_rubin.data.download_rubin_data import (
    download_rubin_data, zenodo_dois
)
from collections import namedtuple

args = namedtuple("Args", ["dataset", "outdir"])
args.dataset = "fea_legacy"
args.outdir = data_dir
download_rubin_data(args)

args.dataset = "bend"
download_rubin_data(args)

In [ ]:
# Compute the intrinsic Zernikes and setup the perturbed telescope builder

fiducial = batoid.Optic.fromYaml("ComCam_r.yaml")
wavelength = 622e-9

zk0 = batoid.zernike(
    fiducial,
    0.0, 0.0,
    wavelength,
    nx=256,
    jmax=28,
    eps=0.61
) * wavelength * 1e6  # -> microns

builder = batoid_rubin.LSSTBuilder(
    fiducial,
    fea_dir=data_dir,
    bend_dir=data_dir
)

In [ ]:
# Define the names of the degrees of freedom
names = []
for component in ["M2", "Cam"]:
    for dof in ["dz", "dx", "dy", "rx", "ry"]:
        names.append(component+"_"+dof)
for i in range(20):
    names.append(f"M1M3_B{i+1}")
for i in range(20):
    names.append(f"M2_B{i+1}")

In [ ]:
# Setup the interactive perturbed telescope delta Zernike evaluator
@ipywidgets.interact(
    idof = ipywidgets.BoundedIntText(value=0, min=0, max=49),
    amplitude = ipywidgets.FloatText(value=1.0, min=-1000, max=1000, step=0.001)
)
def f(idof, amplitude):
    dof = np.zeros(50)
    if idof in [3,4,8,9]:
        dof[idof] = amplitude*3600 # batoid_rubin expects arcsec
        unit = "degrees"
    else:
        dof[idof] = amplitude
        unit = "microns"
    print()
    print(f"Perturbing {names[idof]} by {amplitude} {unit}")
    print("Zernikes changed from fiducial by...")
    perturbed = builder.with_aos_dof(dof).build()
    zk1 = batoid.zernike(
        perturbed,
        0.0, 0.0,
        wavelength,
        nx=256,
        jmax=28,
        eps=0.61
    ) * wavelength * 1e6  # -> microns
    dzk = zk1 - zk0
    print()
    for j in range(4, 17):
        out = f"Z{j:02d} {dzk[j]:8.2f} micron"
        if j < 16:
            out += "          "
            out += f"Z{j+13:02d} {dzk[j+13]:8.2f} micron"
        print(out)

In [ ]:
names[32]